In [54]:
import cv2 as cv
import numpy as np


In [55]:
hough_image = cv.imread('original_imgs/hough.jpg',0)
v_org_image = cv.imread('original_imgs/hough.jpg')
d_org_image = cv.imread('original_imgs/hough.jpg')

sobel_x = [[-1, 0, 1], [-2,0,2], [-1,0,1]]
sobel_y = [[-1, -2, -1], [0,0,0], [1,2,1]]


In [56]:
def getSobelImage(img,sobel):
    height, width= img.shape
    sobelImage=[[0 for col in range(width)] for row in range(height)]
    for x in range(1,height-1):
        for y in range(1,width-1):
            pixel_x =           (sobel[0][0] * img[x-1][y-1]) + \
                            (sobel[0][1] * img[x-1][y]) + \
                            (sobel[0][2] * img[x-1][y+1]) + \
                            (sobel[1][0] * img[x][y-1])   +\
                             (sobel[1][1] * img[x][y])   + \
                             (sobel[1][2] * img[x][y+1]) + \
                             (sobel[2][0] * img[x+1][y-1]) + \
                             (sobel[2][1] * img[x+1][y]) + \
                             (sobel[2][2] * img[x+1][y+1])
            sobelImage[x][y]=pixel_x
    return np.asarray(sobelImage)        
      
def normalizeMatrix(img):
    h,w=img.shape
    currMax=0
    for x in range(0,h):
        for y in range(0,w):
            if (img[x][y]<0):
                img[x][y]= 0-img[x][y]
            if (currMax<img[x][y]):
                currMax=img[x][y]

    for i in range(0,h):
        for j in range(0,w):
            img[i][j]=(img[i][j]/currMax)*255

    return img


In [57]:
def generateHoughLines(image):
    theta = np.deg2rad(np.arange(-90.0, 90.0))
    #theta = np.deg2rad([-90,-60,-45,-30,-15,0,15,30,45,60,90])
    print(image.shape)
    width,height = image.shape
    len_diagonal = int(np.ceil(np.sqrt(width * width + height * height)))
    rhos = np.linspace(-len_diagonal, len_diagonal, len_diagonal * 2.0)

    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    len_thetas = len(theta)

    print(len_diagonal)
    accumulator = np.zeros((2 * len_diagonal, len_thetas), dtype=np.uint64)
    y_idx, x_idx = np.nonzero(image)
    print(len_thetas)

    for i in range(len(x_idx)):
        x = x_idx[i]
        y = y_idx[i]

        for t_idx in range(len_thetas):
            rho = int(round(x * cos_theta[t_idx] + y * sin_theta[t_idx]) + len_diagonal)
            #print(rho)
            accumulator[rho, t_idx] += 1

    return accumulator, theta, rhos

In [58]:
def thresholdSobel(Image):
    img = np.zeros(Image.shape)
    h1,w1 = Image.shape
    for i in range(0,h1):
        for j in range(0,w1):
            if(Image[i][j]>40):
                img[i][j]=255
    return img            

In [59]:

sobelHoughX=getSobelImage(hough_image,sobel_x)
sobelHoughY=getSobelImage(hough_image,sobel_y)
hough_edge_x=normalizeMatrix(sobelHoughX)
hough_edge_y=normalizeMatrix(sobelHoughY)

threshold_sobel_x = thresholdSobel(hough_edge_x)


In [60]:
accumulator, thetas, rhos = generateHoughLines(threshold_sobel_x)

(477, 666)
820
180


/anaconda3/envs/ML_Tensor_Keras/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  import sys


In [61]:
print(accumulator.shape)
#print(accumulator[860,179])

print(np.argmax(accumulator))
idx = np.argmax(accumulator)
r = int(round(idx / accumulator.shape[1]))
rho = rhos[r]
print(rhos)

theta = thetas[idx % accumulator.shape[1]]
print(theta)
print(rho, np.rad2deg(theta))
a=accumulator
nd = np.unravel_index(np.argmax(a, axis=None), a.shape)
print(nd)

#print(accumulator[1244][135])



(1640, 180)
147870
[-820.         -818.99938987 -817.99877974 ...  817.99877974  818.99938987
  820.        ]
0.0
2.501525320317228 0.0
(821, 90)


In [62]:
def hough_peaks1(H, num_peaks):
    indices =  np.argpartition(H.flatten(), -2)[-num_peaks:]
    return np.vstack(np.unravel_index(indices, H.shape)).T

In [63]:
peaks = hough_peaks1(accumulator,60)
v_peaks = peaks[peaks[:,1]>87]
v_peaks = v_peaks[:10,:]
d_peaks = peaks[peaks[:,1]<70]
print(d_peaks)

print(v_peaks)
#theta = thetas[peaks[2][1]]
#theta

hough_lines_draw(v_org_image, v_peaks,rhos, thetas)
hough_lines_draw(d_org_image, d_peaks,rhos, thetas)

[[ 925   53]
 [1146   53]
 [ 856   53]
 [ 860   54]
 [ 855   53]
 [ 933   54]
 [ 785   54]
 [1005   54]
 [ 931   54]
 [ 934   54]
 [1075   54]
 [1078   54]
 [1006   54]
 [ 859   54]
 [ 862   54]
 [1003   54]]
[[1107   89]
 [1007   89]
 [ 908   88]
 [1385   89]
 [1005   88]
 [ 909   88]
 [1196   88]
 [ 907   88]
 [1004   88]
 [1286   88]]


In [64]:
def hough_lines_draw(img, indicies, rhos, thetas):
    for i in range(len(indicies)):
        
        rho = rhos[indicies[i][0]]
        theta = thetas[indicies[i][1]]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))

        cv.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

In [66]:
cv.imwrite("red_line"+'.jpg',v_org_image)
cv.imwrite("blue_lines"+'.jpg',d_org_image)

True